# 6. Vision Transformer

## Introducción

En este Jupyter Notebook, vamos a ajustar un Vision Transformer pre-entrenado (de [🤗 Transformers](https://github.com/huggingface/transformers)) para la clasificación de imágenes. Entrenaremos el modelo usando [PyTorch Lightning ⚡](https://github.com/PyTorchLightning/pytorch-lightning).

HuggingFace 🤗 es una comunidad y biblioteca líder de software de código abierto que ha ganado una atención significativa en los últimos años por sus contribuciones a la democratización de la inteligencia artificial. La biblioteca proporciona modelos pre-entrenados, conjuntos de datos y una suite de herramientas que hacen que sea más fácil para los desarrolladores construir y desplegar aplicaciones de inteligencia artificial. Una de las contribuciones más significativas de HuggingFace es el desarrollo de la biblioteca Transformers, que proporciona una interfaz fácil de usar para trabajar con modelos basados en Transformer, como BERT y GPT.

PyTorch Lightning es una biblioteca de Python de código abierto que proporciona una interfaz de alto nivel para PyTorch. Este framework liviano y de alto rendimiento organiza el código de PyTorch para desacoplar la investigación de la ingeniería, haciendo que los experimentos de Deep Learning sean más fáciles de leer y reproducir.

**Fuente:** Rogge, N. (2021) [Fine-tuning the Vision Transformer on CIFAR-10 with PyTorch Lightning - GitHub](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/VisionTransformer/Fine_tuning_the_Vision_Transformer_on_CIFAR_10_with_PyTorch_Lightning.ipynb).

![vit.png](./docs/Vision_Transformer.png)

## ¿Qué son los Transformers?

La arquitectura Transformer, que fue presentada en el artículo "Attention is All You Need" en 2017, ha revolucionado el mundo del Deep Learning, especialmente en el campo del Procesamiento del Lenguaje Natural. Como un gran modelo de lenguaje basado en la arquitectura GPT-3.5, ChatGPT es la aplicación basada en la arquitectura Transformer más popular del momento. Además de ChatGPT, muchas otras aplicaciones reconocidas, como BERT de Google, la serie GPT de OpenAI y RoBERTa de Facebook, se basan en la arquitectura Transformer para lograr resultados de vanguardia en tareas de NLP. Además, la arquitectura Transformer también ha tenido un gran éxito en el campo de la Visión por Computador, como lo demuestra el éxito de modelos como ViT y DeiT en ImageNet y otros benchmarks de reconocimiento visual.

La principal innovación de la arquitectura Transformer es la combinación del uso de representaciones basadas en atención y un estilo de procesamiento similar al de una red neuronal convolucional (CNN). A diferencia de las redes neuronales convolucionales tradicionales (CNN) que se basan en capas convolucionales para extraer características de las imágenes, los Transformers utilizan mecanismos de atención (auto-atención, atención multi-cabezal) para enfocarse selectivamente en diferentes partes de una secuencia de entrada.

La principal ventaja de los Transformers sobre las CNN tradicionales es que pueden capturar de manera más efectiva las dependencias a largo plazo en los datos. Esto es especialmente útil en tareas de visión por computadora donde una imagen puede contener objetos que están dispersos por toda la imagen, y donde las relaciones entre objetos pueden ser más importantes que los propios objetos. Al atender a diferentes partes de la imagen de entrada, los Transformers pueden aprender eficazmente a extraer estas relaciones y mejorar el rendimiento en tareas como la detección y segmentación de objetos.


**Fuentes:**

+ Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). [Attention is all you need.](https://arxiv.org/abs/1706.03762) - arXiv preprint arXiv:1706.03762. 
+ Dosovitskiy, A., Beyer, L., Kolesnikov, A., Weissenborn, D., Zhai, X., Unterthiner, T., Dehghani, M., Minderer, M., Heigold, G., Gelly, S., Uszkoreit, J., & Houlsby, N. (2020). [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929) - arXiv preprint arXiv:2010.11929.
+ Google Research. (2021). [Vision Transformer and MLP-Mixer Architectures  - GitHub](https://github.com/google-research/vision_transformer)

## Primeros pasos

In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install -q transformers datasets pytorch-lightning

In [1]:
import os
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from src.vit_fine_tune import ViTLightningModule

d:\venvs\no-estruc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create directory where to save the models created
models_dir = "./models"
os.makedirs(models_dir, exist_ok=True)

## Activando CUDA para el procesamiento con GPU

Las GPUs (Graphic Processing Units o Unidades de Procesamiento Gráfico) son procesadores especializados diseñados para manejar los cálculos complejos involucrados en la representación de gráficos e imágenes. Sin embargo, debido a sus capacidades de procesamiento paralelo, también son útiles para una amplia gama de otras aplicaciones, incluyendo el Aprendizaje Automático. A diferencia de las CPU tradicionales, las GPUs pueden manejar muchas tareas más pequeñas simultáneamente, lo que las hace ideales para aplicaciones computacionalmente intensivas.

CUDA es una plataforma de cómputo paralelo y un modelo de programación desarrollado por NVIDIA, diseñado para aprovechar el poder de las GPUs para tareas de cómputo de propósito general. CUDA permite a los desarrolladores escribir programas que se ejecutan en la GPU, aprovechando sus capacidades de procesamiento paralelo para acelerar significativamente el rendimiento.

Para acelerar significativamente el entrenamiento del modelo, utilizaremos la aceleración GPU. Primero comprobaremos si CUDA está disponible en nuestro sistema.

In [3]:
import torch
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {torch.cuda.current_device()}")
        
print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? True
CUDA version: 11.8
ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce GTX 1060 with Max-Q Design


Una vez hecha esta comprobación, realizamos el entrenamiento

## Entrenamiento

TensorBoard es una herramienta de visualización basada en la web proporcionada por TensorFlow para visualizar y analizar varios aspectos de los experimentos de aprendizaje automático.

El comando %load_ext tensorboard carga la extensión de TensorBoard en Jupyter Notebook. El comando %tensorboard --logdir lightning_logs/ inicia TensorBoard y especifica el directorio donde se almacenan los registros, en este caso ./lightning_logs/. TensorBoard lee los eventos y las métricas registradas durante el proceso de entrenamiento y proporciona visualizaciones para analizar el rendimiento del modelo, incluyendo curvas de pérdida y precisión, histogramas de pesos y sesgos, y más.

In [4]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Utilizamos early stopping:

In [5]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

model = ViTLightningModule()

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)

trainer = Trainer(
    accelerator='gpu',
    devices=1,
    callbacks=[
        early_stop_callback
    ]
)

trainer.fit(model)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used

Epoch 9: 100%|██████████| 187/187 [04:23<00:00,  1.41s/it, v_num=0]        


## Validación del modelo

Definimos funciones para realizar comprobaciones sobre el texto:

In [12]:
from tqdm import tqdm

def get_pytorch_predictions_from_dataloader(model, dataloader):
    """
    Get predictions from a Pytorch model on a given dataloader.

    Args:
    -----
    model: PyTorch model
        The model to use for predictions.
    dataloader: PyTorch dataloader
        The dataloader to use for predictions.

    Returns:
    --------
    all_predictions: list
        List of predictions.
    all_targets: list
        List of targets.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    # Move model to device
    model.to(device)

    # Set model to evaluation mode and freeze it
    model.eval()
    model.freeze()

    # Lists to store predictions and targets
    all_predictions = []
    all_targets = []

    # Use a progress bar to show the progress of the predictions
    for batch in tqdm(dataloader):
        images, targets = batch
        images = images.to(device) # Move inputs to the same device as the model
        predictions = model(images)
        # Convert ImageClassifierOutput to tensor
        predictions = predictions.logits
        all_predictions.append(predictions.cpu())
        all_targets.append(targets.cpu())

    # Concatenate all predictions and targets
    all_predictions = torch.cat(all_predictions, dim=0)
    all_targets = torch.cat(all_targets, dim=0)

    return all_predictions, all_targets

In [8]:
# Python base libraries
import os
import glob

DATA_DIR = './src/dataset'
TRAIN_DIR = DATA_DIR + '/training'
VAL_DIR = DATA_DIR + '/validation'

# Data Science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning and Deep Learning libraries
from sklearn.metrics import classification_report
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from transformers import ViTImageProcessor
from torchvision.transforms import (
    RandomResizedCrop,
    RandomHorizontalFlip,
    CenterCrop, 
    Compose, 
    Normalize, 
    Resize, 
    ToTensor
)

def get_vit_metrics(model, train=False):
    """
    Gets the metrics for the ViT model.

    Args:
    -----
    model: PyTorch model
        The ViT model to use for predictions.
    train: bool, optional (default=False)
        Whether to get the metrics for the training set.

    Returns:
    --------
    None
        The classification report for the ViT model is printed
        to the console for both the validation and test sets.
    """
    # Get the image processor and its parameters
    processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
    img_size = processor.size
    img_mean = processor.image_mean
    img_std = processor.image_std

    transform = Compose([
        Resize(img_size['height']),
        CenterCrop(img_size['height']),
        ToTensor(),
        Normalize(mean=img_mean, std=img_std)
    ])

    # Get the classes from the model
    classes = model.id2label.values()

    # Get the dataloaders for the validation set
    val_dataset = ImageFolder(VAL_DIR, transform=transform)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    if train:
        transform_train = Compose([
            RandomResizedCrop(img_size['height']),
            RandomHorizontalFlip(),
            ToTensor(),
            Normalize(mean=img_mean, std=img_std)
        ])
        # Get the dataloader for the training set
        train_dataset = ImageFolder(TRAIN_DIR, transform=transform_train)
        train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=False)

        # Get the classification report for the training set
        predictions, targets = get_pytorch_predictions_from_dataloader(model, train_dataloader)
        print('Training set classification report:')
        print(classification_report(targets, predictions.argmax(dim=1), target_names=classes))

    # Get the classification report for the validation set
    predictions, targets = get_pytorch_predictions_from_dataloader(model, val_dataloader)
    print('Validation set classification report:')
    print(classification_report(targets, predictions.argmax(dim=1), target_names=classes))

    return

In [10]:
LIGHTNING_LOGS_DIR = './lightning_logs'
def load_latest_checkpoint(model_class, logs_dir=LIGHTNING_LOGS_DIR):
    """
    Loads the latest checkpoint from the lightning_logs directory.

    Args:
    -----
    model_class: PyTorch Lightning model class
        The model class to use for loading the checkpoint.
    
    Returns:
    --------
    model: PyTorch Lightning model
        The model loaded from the latest checkpoint.
    """
    version_dirs = glob.glob(os.path.join(logs_dir, 'version_*'))
    latest_version_dir = max(version_dirs, key=os.path.getmtime)
    ckpt_files = glob.glob(os.path.join(latest_version_dir, 'checkpoints', '*.ckpt'))
    latest_ckpt_file = max(ckpt_files, key=os.path.getmtime)
    
    # Load the checkpoint into a new instance of the model class
    model = model_class.load_from_checkpoint(latest_ckpt_file)
    
    return model

In [13]:
# Load best model from the latest checkpoint
best_model = load_latest_checkpoint(ViTLightningModule)
# Get best model metrics
get_vit_metrics(best_model, train=True)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda


100%|██████████| 24/24 [01:12<00:00,  3.02s/it]


Training set classification report:
               precision    recall  f1-score   support

      Bedroom       0.86      0.96      0.91       116
        Coast       0.91      0.98      0.94       260
       Forest       0.98      0.96      0.97       228
      Highway       1.00      0.89      0.94       160
   Industrial       0.97      0.91      0.94       211
  Inside city       0.95      0.93      0.94       208
      Kitchen       0.97      0.99      0.98       110
  Living room       0.98      0.88      0.93       189
     Mountain       0.90      0.99      0.94       274
       Office       0.99      0.99      0.99       115
 Open country       0.96      0.85      0.90       310
        Store       0.95      1.00      0.97       215
       Street       0.89      0.98      0.94       192
       Suburb       0.99      0.98      0.98       141
Tall building       0.96      0.96      0.96       256

     accuracy                           0.95      2985
    macro avg       0.95   

100%|██████████| 47/47 [00:36<00:00,  1.30it/s]

Validation set classification report:
               precision    recall  f1-score   support

      Bedroom       0.86      0.96      0.91       100
        Coast       0.89      0.97      0.93       100
       Forest       1.00      0.89      0.94       100
      Highway       0.98      0.92      0.95       100
   Industrial       0.96      0.81      0.88       100
  Inside city       0.86      0.86      0.86       100
      Kitchen       0.99      0.86      0.92       100
  Living room       0.88      0.87      0.87       100
     Mountain       0.92      1.00      0.96       100
       Office       0.98      0.99      0.99       100
 Open country       0.88      0.81      0.84       100
        Store       0.88      0.96      0.92       100
       Street       0.88      0.98      0.93       100
       Suburb       1.00      0.96      0.98       100
Tall building       0.89      0.97      0.93       100

     accuracy                           0.92      1500
    macro avg       0.92 

En el caso del conjunto de entrenamiento, la matriz de confusión indica que el modelo logró una precisión de alrededor del 95% en la clasificación de las imágenes, lo que significa que la mayoría de las imágenes fueron clasificadas correctamente. La mayoría de las clases tienen una precisión y recall bastante altos, con pocos casos de falsos positivos o negativos. Sin embargo, la clase "Inside city" tuvo una precisión un poco más baja en comparación con las otras clases.

En el conjunto de validación, el modelo obtuvo una precisión del 92%, lo que significa que las imágenes se clasificaron correctamente en la mayoría de los casos. En general, la mayoría de las clases tuvieron una precisión y recall similares a los del conjunto de entrenamiento, pero algunas clases, como "Inside city" y "Open country", tuvieron una precisión ligeramente más baja. Se puede concluir que el modelo tuvo un excelente rendimiento en la clasificación de las imágenes, aunque la precisión y recall varíen según la clase y el conjunto de datos.

En definitiva, el modelo fine-tuneado de Vision Transformer obtiene un rendimiento excelente, el cual está al nivel de los mejores modelos de CNN. Además, no se observa una diferencia significativa en términos de performance entre los conjuntos de entrenamiento y validación, lo que sugiere una gran capacidad de generalización a nuevos datos.

Guardamos el modelo final en `vit_model.pt`.

In [ ]:
# Save the model in the models directory
torch.save(best_model.state_dict(), os.path.join(models_dir, "vit_model.pt"))

## Conclusiones

En resumen, pese a no haber realizado ningún tuneo el Vision Transformer iguala al mejor modelo de CNN del Hackaton y muestra también una gran capacidad de generalización, con un rendimiento de 0.95 de accuracy en training y 0.92 en validation. Podemos deducir que este buen rendimiento del modelo Vision Transformer se debe a su capacidad para capturar las dependencias y las interacciones globales entre las características. Mientras que los modelos CNN tradicionales se basan en operaciones convolucionales y de agrupación para extraer características locales y aplanarlas en un vector, los transformers utilizan mecanismos de autoatención que permiten interacciones globales entre todas las características. Esto permite que los transformers modelen relaciones complejas entre las características e identifiquen dependencias a larga distancia, lo que los hace particularmente efectivos para tareas como la clasificación de imágenes.

Además, la arquitectura jerárquica del Vision Transformer también puede contribuir a su éxito en la tarea de clasificación de estilo artístico. La arquitectura le permite procesar imágenes a múltiples niveles de granularidad, desde características locales hasta la imagen completa. Esto permite que el modelo aprenda representaciones que son más adecuadas para tareas como la clasificación de imágenes, y podría explicar su fuerte rendimiento en este proyecto.